# GraphRAG

법률 문서를 활용한 GraphRAG를 실습합니다.

GraphRAG는 지식 그래프를 활용하여 문서 간의 관계와 계층 구조를 기반으로 검색하는 RAG 시스템입니다.


## Neo4j와 Docling을 활용한 법률문서 Document QA 

In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

LAW_DATA_DIR = "./law"
PROCESSED_LAW_DIR = "./law/markdown"
print(f"  - 원본 법령 경로: {LAW_DATA_DIR}")
print(f"  - 변환 결과 경로: {PROCESSED_LAW_DIR}")

  - 원본 법령 경로: ./law
  - 변환 결과 경로: ./law/markdown


In [2]:
from langchain_neo4j import Neo4jGraph

# Neo4j Desktop 연결 설정
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    database=os.getenv("NEO4J_DATABASE"),
    enhanced_schema=True,
    refresh_schema=True,
)

ValueError: Did not find url, please add an environment variable `NEO4J_URI` which contains it, or pass `url` as a named parameter.

In [ ]:
# 테스트 쿼리 실행
cypher_query = """
CREATE (n:Test {name: "Hello Neo4j"})
RETURN n
"""

graph.query(cypher_query)

In [3]:
def reset_database(graph):
    """
    APOC 없이 데이터베이스 초기화하기
    """
    # 모든 노드와 관계 삭제
    graph.query("MATCH (n) DETACH DELETE n")

    # 모든 제약조건 삭제
    constraints = graph.query("SHOW CONSTRAINTS")
    for constraint in constraints:
        constraint_name = constraint.get("name")
        if constraint_name:
            graph.query(f"DROP CONSTRAINT {constraint_name}")

    # 모든 인덱스 삭제
    indexes = graph.query("SHOW INDEXES")
    for index in indexes:
        index_name = index.get("name")
        index_type = index.get("type")
        if index_name and index_type != "CONSTRAINT":
            graph.query(f"DROP INDEX {index_name}")

    print("데이터베이스가 초기화되었습니다.")


# 데이터베이스 초기화
reset_database(graph)

NameError: name 'graph' is not defined

---

## 2. Docling 한국어 문서 처리

-  **법률문서 PDF 문서 구조를 추출, 변환**
- 주택임대차보호법, 시행령, 시행규칙 (출처: https://www.law.go.kr/)

In [4]:
from glob import glob
from pathlib import Path

# 법령 파일 경로 리스트 확인
LAW_DATA_DIR = Path("./law")
law_files = sorted(LAW_DATA_DIR.glob("*.pdf"))
law_files

[PosixPath('law/근로기준법 시행규칙(고용노동부령)(제00436호)(20250223).pdf'),
 PosixPath('law/근로기준법 시행령(대통령령)(제35276호)(20250223).pdf'),
 PosixPath('law/근로기준법(법률)(제20520호)(20250223).pdf')]

In [6]:
import json
from pathlib import Path

from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import EasyOcrOptions, PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
from tqdm import tqdm

# OCR 설정
pipeline_options = PdfPipelineOptions()
# pipeline_options.ocr_options = EasyOcrOptions(lang=["ko"])  # 한국어 OCR 설정

# DocumentConverter 인스턴스 생성
converter = DocumentConverter(
    format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)}
)

# 각 파일별로 변환 및 처리
for law_file in tqdm(law_files, desc="법률 문서 처리 중"):
    file_path = Path(law_file)
    print(f"\n파일 '{file_path.name}' 변환 중...")

    try:
        # 문서 변환
        result = converter.convert(file_path)

        # 변환 성공 확인
        if result.status == "success":
            # 문서 정보 출력
            print(f"  - 페이지 수: {len(result.document.pages)}")

            # 마크다운으로 내용 추출 (처음 500자만 예시로 표시)
            markdown_content = result.document.export_to_markdown()
            print(f"  - 내용 미리보기: {markdown_content[:500]}...")

            # 결과를 마크다운 파일로 저장
            output_path = LAW_DATA_DIR / f"processed_{file_path.stem}.md"
            with open(output_path, "w", encoding="utf-8") as f:
                f.write(markdown_content)
            print(f"  - 변환된 내용이 {output_path}에 저장되었습니다.")

        else:
            print(f"  - 변환 실패: {result.status}")
    except Exception as e:
        print(f"  - 오류 발생: {str(e)}")

    print("-" * 50)

법률 문서 처리 중:   0%|          | 0/3 [00:00<?, ?it/s]2025-11-10 21:25:07,306 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-10 21:25:07,307 - INFO - Going to convert document batch...
2025-11-10 21:25:07,307 - INFO - Initializing pipeline for StandardPdfPipeline with options hash f9730ffaa6e7f8d4fb0c98c8df3f18cb



파일 '근로기준법 시행규칙(고용노동부령)(제00436호)(20250223).pdf' 변환 중...


2025-11-10 21:25:07,910 - INFO - Auto OCR model selected ocrmac.
2025-11-10 21:25:07,916 - INFO - Accelerator device: 'mps'
2025-11-10 21:25:09,756 - INFO - Accelerator device: 'mps'
2025-11-10 21:25:10,343 - INFO - Processing document 근로기준법 시행규칙(고용노동부령)(제00436호)(20250223).pdf
2025-11-10 21:25:17,091 - INFO - Finished converting document 근로기준법 시행규칙(고용노동부령)(제00436호)(20250223).pdf in 9.79 sec.
<frozen abc>:106: DeprecationWarning: Use BaseMetaSerializer() instead.
/Users/jhj/Desktop/sds_class/.venv/lib/python3.13/site-packages/docling_core/transforms/serializer/markdown.py:490: DeprecationWarning: Field `annotations` is deprecated; use `meta` instead.
  for ann in item.annotations
법률 문서 처리 중:  33%|███▎      | 1/3 [00:09<00:19,  9.80s/it]2025-11-10 21:25:17,108 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-10 21:25:17,110 - INFO - Going to convert document batch...
2025-11-10 21:25:17,110 - INFO - Processing document 근로ᄀ

  - 페이지 수: 4
  - 내용 미리보기: ## 근로기준법 시행규칙

[시행 2025. 2. 23.] [고용노동부령 제436호, 2025. 2. 21., 일부개정]

고용노동부 (임금근로시간정책과 - 근로시간, 휴게) 044-202-7545

고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534

고용노동부 (근로기준정책과 - 임금) 044-202-7548

고용노동부 (여성고용정책과 - 여성) 044-202-7475

고용노동부 (근로기준정책과 - 소년) 044-202-7535

고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7530

고용노동부 (임금근로시간정책과 - 휴일, 연차휴가) 044-202-7973

고용노동부 (임금근로시간정책과 - 유연근로시간제) 044-202-7549

- 제1조(목적) 이 규칙은 「근로기준법」과 같은 법 시행령에서 위임한 사항과 그 시행에 필요한 사항을 규정하는 것을 목 적으로 한다.
- 제2조(손해배상 청구의 신청) 근로자는 「근로...
  - 변환된 내용이 law/processed_근로기준법 시행규칙(고용노동부령)(제00436호)(20250223).md에 저장되었습니다.
--------------------------------------------------

파일 '근로기준법 시행령(대통령령)(제35276호)(20250223).pdf' 변환 중...


2025-11-10 21:25:19,606 - INFO - Finished converting document 근로기준법 시행령(대통령령)(제35276호)(20250223).pdf in 2.50 sec.
/Users/jhj/Desktop/sds_class/.venv/lib/python3.13/site-packages/docling_core/transforms/serializer/markdown.py:490: DeprecationWarning: Field `annotations` is deprecated; use `meta` instead.
  for ann in item.annotations
법률 문서 처리 중:  67%|██████▋   | 2/3 [00:12<00:05,  5.52s/it]2025-11-10 21:25:19,624 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-10 21:25:19,626 - INFO - Going to convert document batch...
2025-11-10 21:25:19,627 - INFO - Processing document 근로기준법(법률)(제20520호)(20250223).pdf


  - 페이지 수: 13
  - 내용 미리보기: ## 근로기준법 시행령

[시행 2025. 2. 23.] [대통령령 제35276호, 2025. 2. 18., 일부개정]

고용노동부 (임금근로시간정책과 - 근로시간, 휴게) 044-202-7545

고용노동부 (근로기준정책과 - 소년) 044-202-7535

고용노동부 (근로기준정책과) 044-202-7546

고용노동부 (근로기준정책과 - 임금) 044-202-7548

고용노동부 (여성고용정책과 - 여성) 044-202-7475

고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534

고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7530

고용노동부 (임금근로시간정책과 - 휴일, 연차휴가) 044-202-7973

고용노동부 (임금근로시간정책과 - 유연근로시간제) 044-202-7549

제1조(목적) 이 영은 「근로기준법」에서 위임한 사항과 그 시행에 필요한 사항을 규정하는 것을 목적으로 한다.

제2조(평균임금의 계...
  - 변환된 내용이 law/processed_근로기준법 시행령(대통령령)(제35276호)(20250223).md에 저장되었습니다.
--------------------------------------------------

파일 '근로기준법(법률)(제20520호)(20250223).pdf' 변환 중...


/Users/jhj/Desktop/sds_class/.venv/lib/python3.13/site-packages/docling_core/types/doc/document.py:2863: DeprecationWarning: ListItem parent must be a list group, creating one on the fly.
  warnings.warn(
2025-11-10 21:25:22,211 - INFO - Finished converting document 근로기준법(법률)(제20520호)(20250223).pdf in 2.59 sec.
/Users/jhj/Desktop/sds_class/.venv/lib/python3.13/site-packages/docling_core/transforms/serializer/markdown.py:490: DeprecationWarning: Field `annotations` is deprecated; use `meta` instead.
  for ann in item.annotations
법률 문서 처리 중: 100%|██████████| 3/3 [00:14<00:00,  4.98s/it]

  - 페이지 수: 21
  - 내용 미리보기: ## 제1장 총칙

- 제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는 국민경제의 발전을 꾀하는 것을 목적으로 한다.

제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. &lt;개정 2018. 3. 20., 2019. 1. 15., 2020. 5. 26.&gt;

1. '근로자'란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 사람을 말한다.
2. '사용자'란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를 말한다.
3. '근로'란 정신노동과 육체노동을 말한다.
4. '근로계약'이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체 결된 계약을 말한다.
5. '임금'이란 사용자가 근로의 대가로 근로자에게 임금, 봉급, 그 밖에 어떠한 명칭으로든지 지급하는 모든 금품을 말한다.
6. '...
  - 변환된 내용이 law/processed_근로기준법(법률)(제20520호)(20250223).md에 저장되었습니다.
--------------------------------------------------


---

## 3. **Knowledge Graph 구축**

### 3.1 데이터 로드


In [7]:
# 마크다운 데이터 로드
import glob
from pathlib import Path

# 처리된 마크다운 파일 목록 가져오기
processed_md_files = sorted(LAW_DATA_DIR.glob("*.md"))

# 각 파일의 내용 로드
law_contents = {}
for md_file in processed_md_files:
    file_path = Path(md_file)
    law_name = file_path.stem.replace("processed_", "")

    with open(file_path, encoding="utf-8") as f:
        content = f.read()

    law_contents[law_name] = content

print(f"로드된 법률 문서: {list(law_contents.keys())}")

로드된 법률 문서: ['근로기준법 시행규칙(고용노동부령)(제00436호)(20250223)', '근로기준법 시행령(대통령령)(제35276호)(20250223)', '근로기준법(법률)(제20520호)(20250223)']


### 3.2 각 법률 문서를 그래프로 변환

- **LaborLawKGExtractor** 클래스는 근로기준법 관련 문서에서 **지식 그래프**를 추출함
- 코드는 법률 문서에서 **장, 조, 항, 호** 등의 계층적 구조를 추출
- Neo4j 데이터베이스에 **노드와 관계**를 생성하여 법률 온톨로지를 구축
- 추출된 구조는 **GraphDocument** 객체로 변환되어 Neo4j에 저장


In [8]:
import os
import re

from langchain_neo4j import Neo4jGraph
from langchain_neo4j.graphs.graph_document import GraphDocument, Node, Relationship
from tqdm import tqdm


class LaborLawKGExtractor:
    """근로기준법 관련 문서에서 지식 그래프를 추출하는 클래스"""

    def __init__(self, law_contents):
        """
        초기화 함수

        Args:
            law_contents (dict): 법률 문서 내용을 담은 딕셔너리
        """
        self.law_contents = law_contents
        self.node_dict = {}  # 노드 ID를 키로 사용하여 생성된 노드 객체를 저장
        self.relationships = []
        # Neo4j 데이터베이스 연결 설정
        self.graph = Neo4jGraph(
            url=os.getenv("NEO4J_URI"),
            username=os.getenv("NEO4J_USERNAME"),
            password=os.getenv("NEO4J_PASSWORD"),
            database=os.getenv("NEO4J_DATABASE"),
            enhanced_schema=True,
            refresh_schema=True,
        )

    def extract_structure(self):
        """
        문서에서 장, 조, 항, 호 등의 계층적 구조를 추출

        Returns:
            dict: 법률별로 계층적 구조를 가진 딕셔너리
        """
        all_laws = {}

        # 각 법률 문서에 대해 처리
        for law_name, content in self.law_contents.items():
            # 문서 구조 추출
            law_structure = self.extract_sections(content, law_name)
            all_laws[law_name] = law_structure

        return all_laws

    def extract_sections(self, content, law_name):
        """
        문서에서 섹션(장, 조 등)을 계층적으로 추출

        Args:
            content (str): 법률 문서 내용
            law_name (str): 법률 문서 이름

        Returns:
            dict: 계층적 구조를 가진 법률 문서 딕셔너리
        """
        law_structure = {
            "name": law_name,  # 법률 이름
            "type": "law",  # 노드 타입
            "full_text": content,  # 전체 법률 텍스트
            "chapters": [],  # 장/절 목록을 저장할 리스트
        }

        # 제목 패턴 (# 또는 ## 로 시작하는 라인) - 마크다운 형식의 제목 인식
        title_pattern = r"^(#+)\s+(.+)$"

        # 조문 패턴 (제X조) - 법률 조문 형식 인식 (예: 제1조(목적) 이 법은...)
        article_pattern = r"제(\d+)조\(([^)]+)\)\s+(.+)"

        # 항 패턴 (①, ②, ③ 등으로 시작) - 법률 항 형식 인식
        paragraph_pattern = r"[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮]\s+(.+)"

        # 호 패턴 (1., 2., 3. 등으로 시작) - 법률 호 형식 인식
        subparagraph_pattern = r'[-\s]*(\d+)\.\s+[\'"]?(.+?)[\'"]?$'

        lines = content.split("\n")  # 줄 단위로 분리
        current_chapter = None  # 현재 처리 중인 장/절
        current_article = None  # 현재 처리 중인 조문
        current_paragraph = None  # 현재 처리 중인 항

        for line in lines:
            # 장/절 매칭 - 마크다운 제목 형식 확인
            chapter_match = re.match(title_pattern, line)
            if chapter_match:
                level = len(chapter_match.group(1))  # # 개수로 제목 레벨 결정
                title = chapter_match.group(2).strip()  # 제목 텍스트 추출

                if "장" in title or "절" in title:  # 장 또는 절이 포함된 제목인 경우
                    current_chapter = {
                        "title": title,  # 장/절 제목
                        "type": "chapter",  # 노드 타입
                        "full_text": title,  # 전체 텍스트 (초기값은 제목)
                        "articles": [],  # 조문 목록을 저장할 리스트
                    }
                    law_structure["chapters"].append(current_chapter)  # 법률 구조에 장/절 추가
                    current_article = None  # 새 장/절로 이동했으므로 현재 조문 초기화
                    current_paragraph = None  # 현재 항 초기화
                continue

            # 조문 매칭 - 법률 조문 형식 확인
            article_match = re.search(article_pattern, line)
            if article_match:
                article_num = article_match.group(1)  # 조문 번호 추출
                article_title = article_match.group(2)  # 조문 제목 추출
                article_content = article_match.group(3)  # 조문 내용 추출

                article_full_title = f"제{article_num}조({article_title})"  # 전체 조문 제목 생성

                current_article = {
                    "number": article_num,  # 조문 번호
                    "title": article_full_title,  # 조문 전체 제목
                    "content": article_content,  # 조문 내용
                    "full_text": f"{article_full_title} {article_content}",  # 조문 전체 텍스트
                    "type": "article",  # 노드 타입
                    "paragraphs": [],  # 항 목록을 저장할 리스트
                }

                # 조문이 속한 장이 없으면 기본 장 생성 (장이 명시되지 않은 조문을 위함)
                if current_chapter is None:
                    current_chapter = {
                        "title": "기본",  # 기본 장 제목
                        "type": "chapter",  # 노드 타입
                        "full_text": "기본",  # 전체 텍스트
                        "articles": [],  # 조문 목록
                    }
                    law_structure["chapters"].append(current_chapter)  # 법률 구조에 기본 장 추가

                current_chapter["articles"].append(current_article)  # 현재 장에 조문 추가
                current_paragraph = None  # 새 조문으로 이동했으므로 현재 항 초기화

                # 장의 full_text 업데이트 - 조문 정보 추가
                current_chapter["full_text"] += f"\n{current_article['full_text']}"
                continue

            # 항 매칭 - 법률 항 형식 확인
            paragraph_match = re.match(paragraph_pattern, line)
            if paragraph_match and current_article:  # 현재 조문이 있는 경우에만 항 처리
                paragraph_content = paragraph_match.group(1)  # 항 내용 추출

                current_paragraph = {
                    "content": paragraph_content,  # 항 내용
                    "full_text": paragraph_content,  # 항 전체 텍스트
                    "type": "paragraph",  # 노드 타입
                    "subparagraphs": [],  # 호 목록을 저장할 리스트
                }

                current_article["paragraphs"].append(current_paragraph)  # 현재 조문에 항 추가

                # 조문의 full_text 업데이트 - 항 정보 추가
                current_article["full_text"] += f"\n{paragraph_content}"
                # 장의 full_text 업데이트 - 항 정보 추가
                current_chapter["full_text"] += f"\n{paragraph_content}"
                continue

            # 호 매칭 - 법률 호 형식 확인
            subparagraph_match = re.match(subparagraph_pattern, line)
            if subparagraph_match and current_paragraph:  # 현재 항이 있는 경우에만 호 처리
                subparagraph_num = subparagraph_match.group(1)  # 호 번호 추출
                subparagraph_content = subparagraph_match.group(2)  # 호 내용 추출

                subparagraph = {
                    "number": subparagraph_num,  # 호 번호
                    "content": subparagraph_content,  # 호 내용
                    "full_text": f"{subparagraph_num}. {subparagraph_content}",  # 호 전체 텍스트
                    "type": "subparagraph",  # 노드 타입
                }

                current_paragraph["subparagraphs"].append(subparagraph)  # 현재 항에 호 추가

                # 항의 full_text 업데이트 - 호 정보 추가
                current_paragraph["full_text"] += f"\n{subparagraph['full_text']}"
                # 조문의 full_text 업데이트 - 호 정보 추가
                current_article["full_text"] += f"\n{subparagraph['full_text']}"
                # 장의 full_text 업데이트 - 호 정보 추가
                current_chapter["full_text"] += f"\n{subparagraph['full_text']}"

        return law_structure

    def create_knowledge_graph(self):
        """
        지식 그래프 생성 - 법, 장, 조, 항, 호 간의 계층적 관계 구축

        Returns:
            bool: 그래프 생성 성공 여부
        """
        # 법률 구조 추출
        all_laws = self.extract_structure()

        # 노드 및 관계 생성
        for law_name, law_structure in tqdm(all_laws.items(), desc="법률 온톨로지 구축 중"):
            # 법률 노드 생성
            law_id = f"law_{law_name}"  # 법률 노드 고유 ID 생성
            law_node = Node(
                id=law_id,  # 노드 ID
                type="Law",  # 노드 타입 (법률)
                properties={
                    "name": law_name,  # 법률 이름
                    "full_text": law_structure["full_text"],  # 법률 전체 텍스트
                },
            )
            self.node_dict[law_id] = law_node  # 노드 사전에 법률 노드 추가

            # 장 노드 생성 및 법률과 연결
            for chapter in law_structure["chapters"]:
                chapter_id = f"chapter_{law_name}_{chapter['title']}"  # 장 노드 고유 ID 생성
                chapter_node = Node(
                    id=chapter_id,  # 노드 ID
                    type="Chapter",  # 노드 타입 (장)
                    properties={
                        "title": chapter["title"],  # 장 제목
                        "full_text": chapter["full_text"],  # 장 전체 텍스트
                    },
                )
                self.node_dict[chapter_id] = chapter_node  # 노드 사전에 장 노드 추가

                # 법률과 장 연결 - CONTAINS 관계 생성
                self.relationships.append(
                    Relationship(
                        source=self.node_dict[law_id],  # 출발 노드 (법률)
                        target=self.node_dict[chapter_id],  # 도착 노드 (장)
                        type="CONTAINS",  # 관계 타입 (포함)
                        properties={},  # 관계 속성 (없음)
                    )
                )

                # 조문 노드 생성 및 장과 연결
                prev_article_node = None  # 이전 조문 노드 추적용 변수

                for article in chapter["articles"]:
                    article_id = f"article_{law_name}_{article['title']}"  # 조문 노드 고유 ID 생성
                    article_node = Node(
                        id=article_id,  # 노드 ID
                        type="Article",  # 노드 타입 (조문)
                        properties={
                            "title": article["title"],  # 조문 제목
                            "content": article["content"],  # 조문 내용
                            "number": article["number"],  # 조문 번호
                            "full_text": article["full_text"],  # 조문 전체 텍스트
                        },
                    )
                    self.node_dict[article_id] = article_node  # 노드 사전에 조문 노드 추가

                    # 장과 조문 연결 - CONTAINS 관계 생성
                    self.relationships.append(
                        Relationship(
                            source=self.node_dict[chapter_id],  # 출발 노드 (장)
                            target=self.node_dict[article_id],  # 도착 노드 (조문)
                            type="CONTAINS",  # 관계 타입 (포함)
                            properties={},  # 관계 속성 (없음)
                        )
                    )

                    # 이전 조문과 현재 조문 간의 NEXT_TO 관계 생성
                    if prev_article_node:
                        self.relationships.append(
                            Relationship(
                                source=prev_article_node,  # 출발 노드 (이전 조문)
                                target=self.node_dict[article_id],  # 도착 노드 (현재 조문)
                                type="NEXT_TO",  # 관계 타입 (다음)
                                properties={},  # 관계 속성 (없음)
                            )
                        )

                    # 현재 조문을 이전 조문으로 설정
                    prev_article_node = self.node_dict[article_id]

                    # 항 노드 생성 및 조문과 연결
                    prev_paragraph_node = None  # 이전 항 노드 추적용 변수

                    for i, paragraph in enumerate(article["paragraphs"]):
                        paragraph_id = f"paragraph_{law_name}_{article['title']}_{i + 1}"  # 항 노드 고유 ID 생성
                        paragraph_node = Node(
                            id=paragraph_id,  # 노드 ID
                            type="Paragraph",  # 노드 타입 (항)
                            properties={
                                "content": paragraph["content"],  # 항 내용
                                "number": i + 1,  # 항 번호 (인덱스 기반)
                                "full_text": paragraph["full_text"],  # 항 전체 텍스트
                            },
                        )
                        self.node_dict[paragraph_id] = paragraph_node  # 노드 사전에 항 노드 추가

                        # 조문과 항 연결 - CONTAINS 관계 생성
                        self.relationships.append(
                            Relationship(
                                source=self.node_dict[article_id],  # 출발 노드 (조문)
                                target=self.node_dict[paragraph_id],  # 도착 노드 (항)
                                type="CONTAINS",  # 관계 타입 (포함)
                                properties={},  # 관계 속성 (없음)
                            )
                        )

                        # 이전 항과 현재 항 간의 NEXT_TO 관계 생성
                        if prev_paragraph_node:
                            self.relationships.append(
                                Relationship(
                                    source=prev_paragraph_node,  # 출발 노드 (이전 항)
                                    target=self.node_dict[paragraph_id],  # 도착 노드 (현재 항)
                                    type="NEXT_TO",  # 관계 타입 (다음)
                                    properties={},  # 관계 속성 (없음)
                                )
                            )

                        # 현재 항을 이전 항으로 설정
                        prev_paragraph_node = self.node_dict[paragraph_id]

                        # 호 노드 생성 및 항과 연결
                        prev_subparagraph_node = None  # 이전 호 노드 추적용 변수

                        for subparagraph in paragraph["subparagraphs"]:
                            subparagraph_id = f"subparagraph_{law_name}_{article['title']}_{i + 1}_{subparagraph['number']}"  # 호 노드 고유 ID 생성
                            subparagraph_node = Node(
                                id=subparagraph_id,  # 노드 ID
                                type="Subparagraph",  # 노드 타입 (호)
                                properties={
                                    "content": subparagraph["content"],  # 호 내용
                                    "number": subparagraph["number"],  # 호 번호
                                    "full_text": subparagraph["full_text"],  # 호 전체 텍스트
                                },
                            )
                            self.node_dict[subparagraph_id] = (
                                subparagraph_node  # 노드 사전에 호 노드 추가
                            )

                            # 항과 호 연결 - CONTAINS 관계 생성
                            self.relationships.append(
                                Relationship(
                                    source=self.node_dict[paragraph_id],  # 출발 노드 (항)
                                    target=self.node_dict[subparagraph_id],  # 도착 노드 (호)
                                    type="CONTAINS",  # 관계 타입 (포함)
                                    properties={},  # 관계 속성 (없음)
                                )
                            )

                            # 이전 호와 현재 호 간의 NEXT_TO 관계 생성
                            if prev_subparagraph_node:
                                self.relationships.append(
                                    Relationship(
                                        source=prev_subparagraph_node,  # 출발 노드 (이전 호)
                                        target=self.node_dict[
                                            subparagraph_id
                                        ],  # 도착 노드 (현재 호)
                                        type="NEXT_TO",  # 관계 타입 (다음)
                                        properties={},  # 관계 속성 (없음)
                                    )
                                )

                            # 현재 호를 이전 호로 설정
                            prev_subparagraph_node = self.node_dict[subparagraph_id]

        # GraphDocument 객체 생성 - Neo4j에 저장하기 위한 형식
        nodes = list(self.node_dict.values())  # 모든 노드 목록
        graph_doc = GraphDocument(
            nodes=nodes,  # 노드 목록
            relationships=self.relationships,  # 관계 목록
        )

        # 기존 데이터 삭제 - 데이터베이스 초기화
        # self.graph.query("MATCH (n) DETACH DELETE n")

        # 생성된 GraphDocument를 Neo4j 데이터베이스에 저장
        self.graph.add_graph_documents([graph_doc])

        print(f"총 노드 수: {len(self.node_dict)}")
        print(f"총 관계 수: {len(self.relationships)}")
        print("법률 온톨로지 구축 완료!")

        return True


# 마크다운 파일에서 법률 문서 구조 추출하는 함수
def extract_law_structure(law_contents):
    """
    법률 문서 구조를 추출하는 함수

    Args:
        law_contents (dict): 법률 문서 내용을 담은 딕셔너리

    Returns:
        dict: 법률별로 계층적 구조를 가진 딕셔너리
    """
    extractor = LaborLawKGExtractor(law_contents)  # 추출기 객체 생성
    all_laws = extractor.extract_structure()  # 구조 추출 메서드 호출
    return all_laws


# 법률 문서 구조 추출
all_laws = extract_law_structure(law_contents)

# 추출된 문서 구조 확인
print(f"추출된 법률 문서 수: {len(all_laws)}")
first_law_name = list(all_laws.keys())[0]  # 첫 번째 법률 이름 가져오기
print(f"첫 번째 법률: {first_law_name}")
print(f"첫 번째 법률의 장 수: {len(all_laws[first_law_name]['chapters'])}")

ValueError: Did not find url, please add an environment variable `NEO4J_URI` which contains it, or pass `url` as a named parameter.

In [ ]:
# 지식 그래프 구축
extractor = LaborLawKGExtractor(law_contents)
extractor.create_knowledge_graph()

In [ ]:
# 지식 그래프 확인
graph_db = extractor.graph

# 모든 노드 조회
graph_db.query("MATCH (n) RETURN count(n)")

### 3.3 법령 간의 관계를 추가

- **법률-시행령**, **시행령-시행규칙** 간의 관계를 정의
- 각 관계는 **HAS_DECREE**, **HAS_RULE** 유형으로 Neo4j 쿼리문을 통해 생성

In [9]:
# 법률과 시행령 간의 관계 추가
law_decree_query = """
MATCH (law:Law), (decree:Law)
WHERE law.name CONTAINS '근로기준법(법률)'
AND decree.name CONTAINS '근로기준법 시행령(대통령령)'
CREATE (law)-[r:HAS_DECREE]->(decree)
RETURN count(r) as relationships_created
"""

# 시행령과 시행규칙 간의 관계 추가
decree_rule_query = """
MATCH (decree:Law), (rule:Law)
WHERE decree.name CONTAINS '근로기준법 시행령(대통령령)'
AND rule.name CONTAINS '근로기준법 시행규칙(고용노동부령'
CREATE (decree)-[r:HAS_RULE]->(rule)
RETURN count(r) as relationships_created
"""

# 쿼리 실행 및 결과 확인
law_decree_result = graph_db.query(law_decree_query)
decree_rule_result = graph_db.query(decree_rule_query)

print(f"법률-시행령 관계 생성: {law_decree_result[0]['relationships_created']}개")
print(f"시행령-시행규칙 관계 생성: {decree_rule_result[0]['relationships_created']}개")

NameError: name 'graph_db' is not defined

---

## 4. **Graph RAG 구현**

### 4.1 벡터 저장소에 인덱싱

- 법률 조문을 위한 **벡터 인덱스**를 추가
- **벡터 임베딩** 생성 및 Neo4j 저장 기능 구현

#### 1) **벡터 임베딩 모델** 설정

- 뉴스 본문의 벡터화 및 저장을 위한 기초 작업
- 임베딩 모델 설정은 벡터 검색 성능에 직접적 영향을 미침

In [ ]:
# OpenAI 임베딩 모델 초기화(OpenRouter 활용)
embeddings = create_openrouter_embedding(model="text-embedding-3-small")

#### 2) **벡터 인덱스** 생성

- 각 조문 노드의 **content_embedding** 필드에 적용하고 벡터 인덱스를 개별 생성함 (각 레이블 별로 별도 인덱스 생성만 가능)
- 벡터 차원을 **1536차원**으로 설정하여 OpenAI의 text-embedding-3-small 모델과 호환되도록 함

In [ ]:
# 법률/시행령/시행규칙 조항 벡터 인덱스 생성
create_law_index_query = """
CREATE VECTOR INDEX law_article_embeddings IF NOT EXISTS
FOR (n:Article)
ON n.content_embedding
OPTIONS {indexConfig: {
  `vector.dimensions`: 1536,
  `vector.similarity_function`: 'cosine'
}}
"""

# 모든 벡터 인덱스 생성 쿼리 실행
graph.query(create_law_index_query)

In [ ]:
# 벡터 인덱스 확인
check_vector_index_query = """
SHOW VECTOR INDEXES
"""
vector_indexes = graph.query(check_vector_index_query)
for index in vector_indexes:
    # 벡터 인덱스 정보 출력
    print(f"Index Name: {index['name']}")
    print(f"Type: {index['type']}")
    print(f"Property Key: {index['properties']}")
    print("-" * 40)

#### 3) **임베딩 생성 및 저장**

- 텍스트에 대해 **OpenAI 임베딩**을 생성하는 과정 수행
- 빈 문자열인 경우 처리를 **건너뛰는** 예외 처리 포함
- 생성된 임베딩을 `db.create.setNodeVectorProperty` 프로시저를 통해 **content_embedding** 속성으로 저장

In [ ]:
# 법률 조항 데이터 가져오기
law_query = """
MATCH (a:Article)
WHERE a.content IS NOT NULL
RETURN a.id AS id, a.title AS title, a.content AS content
"""
law_articles = graph.query(law_query)

# 배치 크기 설정
BATCH_SIZE = 100

# 임베딩 생성 및 저장 (배치 처리)
for i in range(0, len(law_articles), BATCH_SIZE):
    batch = law_articles[i : i + BATCH_SIZE]
    batch_texts = []
    batch_ids = []

    # 배치 데이터 준비
    for article in batch:
        content_text = f"{article['title']}\n\n{article['content']}"
        if content_text.strip():  # 빈 문자열 확인
            batch_texts.append(content_text)
            batch_ids.append(article["id"])

    try:
        if batch_texts:
            # 배치 단위로 OpenAI 임베딩 생성
            batch_embeddings = embeddings.embed_documents(batch_texts)

            # UNWIND를 사용한 배치 업데이트
            batch_data = [
                {"id": article_id, "embedding": embedding_vector}
                for article_id, embedding_vector in zip(batch_ids, batch_embeddings, strict=False)
            ]

            batch_update_query = """
            UNWIND $batch AS item
            MATCH (a:Article {id: item.id})
            CALL db.create.setNodeVectorProperty(a, 'content_embedding', item.embedding)
            RETURN count(a) as updated
            """

            result = graph.query(batch_update_query, params={"batch": batch_data})
            print(
                f"배치 처리 완료: {i + 1}~{min(i + len(batch_texts), len(law_articles))} / {len(law_articles)}, 업데이트됨: {result[0]['updated']}"
            )
    except Exception as e:
        print(f"배치 임베딩 생성 실패 (배치 인덱스 {i}): {str(e)}")

print(f"법률 조항 임베딩 업데이트 완료!! 총 {len(law_articles)}개 처리")

#### 3) **임베딩 생성 및 저장**

- 텍스트에 대해 **OpenAI 임베딩**을 생성하는 과정 수행
- 빈 문자열인 경우 처리를 **건너뛰는** 예외 처리 포함
- 생성된 임베딩을 `db.create.setNodeVectorProperty` 프로시저를 통해 **content_embedding** 속성으로 저장

### 4.2 RAG 시스템 구현

#### 1) **Neo4j Graph DB 검색 설정**


In [ ]:
from langchain_neo4j import Neo4jVector

# 임베딩 모델 설정
embeddings = create_embedding_model(model="text-embedding-3-small")

# Neo4j 데이터베이스에 이미 생성된 벡터 인덱스에 연결하는 Neo4jVector 인스턴스 생성
vector_store = Neo4jVector.from_existing_index(
    embeddings,  # 사용할 임베딩 모델 지정
    url=os.getenv("NEO4J_URI"),  # Neo4j 데이터베이스 연결 URI (환경 변수에서 가져옴)
    username=os.getenv("NEO4J_USERNAME"),  # Neo4j 데이터베이스 사용자 이름
    password=os.getenv("NEO4J_PASSWORD"),  # Neo4j 데이터베이스 비밀번호
    database=os.getenv("NEO4J_DATABASE"),  # Neo4j 데이터베이스 이름
    index_name="law_article_embeddings",  # 법률 데이터용 벡터 인덱스 이름
    node_label="Article",  # 법률 조항 노드 레이블
    text_node_property="content",  # 텍스트 검색 시 반환할 노드의 속성 (법률 내용)
    embedding_node_property="content_embedding",  # 임베딩이 저장된 속성 이름
)

In [ ]:
# 벡터 검색 테스트
query = "연차휴가는 연간 몇일을 부여해야 하나요?"
results = vector_store.similarity_search_with_score(
    query=query,
    k=5,
    return_embeddings=False,
)

print(f"검색어: '{query}'에 대한 결과")
print("-" * 50)

for i, (doc, score) in enumerate(results):
    similarity = 1 - score  # 코사인 거리를 유사도로 변환
    print(f"\n결과 #{i + 1} (유사도: {similarity:.4f})")
    print(f"제목: {doc.metadata.get('title', '제목 없음')}")
    print(f"내용 미리보기: {doc.page_content[:150]}...")

#### 2) Neo4j 에 저장된 벡터 검색 기반 RAG 구현

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 법률 데이터를 위한 RAG 프롬프트 템플릿 정의
template = """
당신은 한국 법률 전문가 AI 비서입니다.
제공된 법률 조항 내용을 바탕으로 질문에 정확하게 답변해 주세요. 출처를 반드시 표기해 주세요. (예: 출처: 법률 조항 제목)
법률 조항에서 찾을 수 없는 정보에 대해서는 솔직하게 모른다고 답변하세요.
법적 조언이 필요한 경우에는 전문 법률 상담을 권유하세요.

참고할 법률 조항:
{context}

질문: {question}

답변:
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# 법률 데이터 RAG 체인 구성
llm = create_openrouter_llm(model="openai/gpt-4.1", temperature=0)
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

law_rag_chain = (
    {"question": RunnablePassthrough(), "context": retriever} | prompt | llm | StrOutputParser()
)


# 법률 RAG 실행 예시
def law_assistant(query):
    """법률 질문에 대한 답변을 제공하는 함수"""
    response = law_rag_chain.invoke(query)
    print(f"질문: {query}\n\n답변: {response}\n{'-' * 70}")
    return response


# 테스트 질문 예시 - 근로기준법 관련
test_questions = [
    "법정 근로시간은 어떻게 되나요?",
    "주휴수당의 계산 방법과 지급 기준은 무엇인가요?",
    "해고 예고 제도의 내용과 예외는 무엇인가요?",
    "연차유급휴가 발생 요건과 일수 계산 방법은 어떻게 되나요?",
    "직장 내 괴롭힘 금지에 관한 규정은 무엇인가요?",
    "임금 체불 시 사업주에 대한 제재 규정은 어떻게 되나요?",
    "출산 전후 휴가와 육아휴직 제도의 주요 내용은 무엇인가요?",
    "근로계약서 작성 의무와 필수 기재사항은 무엇인가요?",
]

# 테스트 질문 실행
for question in test_questions:
    law_assistant(question)

#### 3) 지식 그래프로 강화된 GraphRAG 구현

In [ ]:
def kg_enhanced_law_rag(question):
    """지식 그래프 정보로 강화된 법률 RAG 시스템"""
    try:
        # 1. 벡터 검색으로 관련 법률 문서 찾기
        docs = vector_store.similarity_search(question, k=5)

        # 검색된 문서의 ID 추출
        doc_ids = []
        for doc in docs:
            if "id" in doc.metadata:
                doc_ids.append(doc.metadata["id"])
            elif "title" in doc.metadata:
                doc_ids.append(doc.metadata["title"])

        # 검색 결과가 없는 경우 처리
        if not doc_ids:
            return {
                "query": question,
                "result": "관련 법률 조항을 찾을 수 없습니다. 다른 질문을 시도해보세요.",
                "intermediate_steps": [],
            }

        # 2. 그래프 검색: 가변 경로를 사용하여 관련 조문과 그 전후 조문 찾기 (2단계 깊이까지)
        cypher_query = """
        // 벡터 검색 결과와 일치하는 법률 조문 찾기
        MATCH (article:Article)
        WHERE article.id IN $doc_ids OR article.title IN $doc_ids

        // 가변 경로를 사용하여 1~2단계 깊이의 조문 찾기
        MATCH (article)-[r*1..2]->(related:Article)

        // 결과 수집 및 가공
        WITH article, related, r
        WITH article,
             related,
             size(r) AS path_length,
             [rel IN r | type(rel)] AS relationship_types

        // 최종 결과 반환
        RETURN article.id AS article_id,
               article.title AS title,
               article.content AS content,
               COLLECT(DISTINCT {
                   id: related.id,
                   title: related.title,
                   content: related.content,
                   path_length: path_length,
                   relationships: relationship_types
               }) AS relatedArticles
        """

        # 그래프 검색 실행 및 결과 처리
        graph_results = graph.query(cypher_query, {"doc_ids": doc_ids})

        # 3. 그래프 정보를 텍스트로 변환
        kg_context = ""
        for record in graph_results:
            kg_context += f"조문: {record['title']}\n내용: {record['content']}\n\n"

            # 관련 조문 정보 추가 (경로 길이에 따라 구분)
            kg_context += "관련 조문:\n"

            # 1단계와 2단계 관계를 구분하여 표시
            level1_articles = [rel for rel in record["relatedArticles"] if rel["path_length"] == 1]
            level2_articles = [rel for rel in record["relatedArticles"] if rel["path_length"] == 2]

            # 1단계 관련 조문 표시
            kg_context += "직접 관련 조문:\n"
            for article in level1_articles:
                rel_type = article["relationships"][0] if article["relationships"] else "관계 없음"
                kg_context += f"- {article['title']} (관계: {rel_type})\n  {article['content']}\n\n"

            # 2단계 관련 조문 표시
            kg_context += "간접 관련 조문:\n"
            for article in level2_articles:
                rel_types = (
                    " -> ".join(article["relationships"])
                    if article["relationships"]
                    else "관계 없음"
                )
                kg_context += (
                    f"- {article['title']} (관계 경로: {rel_types})\n  {article['content']}\n\n"
                )

        # 4. 원본 문서 내용 가져오기
        doc_context = "\n".join(
            [f"{doc.metadata.get('title', '제목 없음')}: {doc.page_content}" for doc in docs]
        )

        # 5. 통합 컨텍스트 생성
        combined_context = f"법률 문서 정보:\n{doc_context}\n\n법률 지식 그래프 정보:\n{kg_context}"

        # 6. 프롬프트 템플릿 정의
        kg_template = """
        당신은 근로기준법에 대한 전문 지식을 갖춘 법률 전문가입니다.
        제공된 법률 조문과 지식 그래프 정보를 바탕으로 질문에 정확하게 답변해 주세요.

        지식 그래프는 법률 조문 간의 관계와 연결성을 보여줍니다.
        이 관계 정보를 활용하여 더 정확하고 포괄적인 법률 해석을 제공하세요.

        법률 조문이나 지식 그래프에서 찾을 수 없는 정보에 대해서는 솔직하게 모른다고 답변하세요.
        답변은 법률 용어를 적절히 사용하되, 일반인도 이해할 수 있도록 명확하게 작성해 주세요.

        참고할 정보:
        {context}

        질문: {question}

        답변:
        """

        kg_prompt = PromptTemplate(template=kg_template, input_variables=["context", "question"])

        # 7. RAG 체인 구성 및 실행
        rag_chain = kg_prompt | llm | StrOutputParser()
        result = rag_chain.invoke({"question": question, "context": combined_context})

        # 8. 중간 단계 정보 포함하여 결과 반환
        intermediate_steps = [
            {"query": cypher_query},
            {"context": [dict(record) for record in graph_results]},
        ]

        return {"query": question, "result": result, "intermediate_steps": intermediate_steps}

    except Exception as e:
        # 오류 처리 및 디버깅 정보 반환
        return {
            "query": question,
            "result": f"검색 중 오류가 발생했습니다: {str(e)}",
            "error": str(e),
        }


# 실행 테스트
result = kg_enhanced_law_rag("법정 근로시간은 어떻게 되나요?")
print(result["result"])

In [ ]:
from pprint import pprint

# 실행 테스트
result = kg_enhanced_law_rag("법정 근로시간 연장이 가능한 특별한 사정에 대해서 설명해주세요.")
print(result["result"])
pprint(result["intermediate_steps"])

### 4.3 전문 검색(fulltext) 결합

#### 1) Nori 분석기를 위한 인덱스 설정

- 한국어 텍스트를 위한 Nori 분석기를 사용

In [ ]:
cypher_query = """
// 한국어 법률 조문을 위한 전체 텍스트 인덱스 생성
CREATE FULLTEXT INDEX article_fulltext IF NOT EXISTS
FOR (a:Article) ON EACH [a.title, a.content]
OPTIONS {
  indexConfig: {
    `fulltext.analyzer`: 'nori',  // 한국어 분석기
    `fulltext.eventually_consistent`: true  // 성능 향상을 위한 설정
  }
}
"""

graph.query(cypher_query)

In [ ]:
# Neo4j의 전문 검색 활용
fulltext_query = """
CALL db.index.fulltext.queryNodes("article_fulltext", $query)
YIELD node, score
RETURN node.id AS id, node.title AS title, node.content AS content, score
ORDER BY score DESC
LIMIT 3
"""
question = "법정 근로시간 연장이 가능한 특별한 사정에 대해서 설명해주세요."
fulltext_results = graph.query(fulltext_query, {"query": question})

# 결과 출력
for record in fulltext_results:
    print(f"ID: {record['id']}")
    print(f"제목: {record['title']}")
    print(f"내용: {record['content']}")
    print(f"점수: {record['score']}")
    print("-" * 50)

#### 2) 전문 검색과 벡터 검색을 결합한 Hybrid RAG

In [ ]:
def hybrid_kg_enhanced_law_rag(question):
    """전문 검색, 벡터 검색, 지식 그래프를 결합한 법률 RAG 시스템"""
    try:
        # 1. 벡터 검색으로 관련 법률 문서 찾기
        vector_docs = vector_store.similarity_search(question, k=3)

        # 검색된 문서의 ID 추출
        doc_ids = []
        for doc in vector_docs:
            if "id" in doc.metadata:
                doc_ids.append(doc.metadata["id"])
            elif "title" in doc.metadata:
                doc_ids.append(doc.metadata["title"])

        # 2. 전문 검색으로 관련 법률 문서 찾기
        # Neo4j의 전문 검색 활용
        fulltext_query = """
        CALL db.index.fulltext.queryNodes("article_fulltext", $query)
        YIELD node, score
        RETURN node.id AS id, node.title AS title, node.content AS content, score
        ORDER BY score DESC
        LIMIT 3
        """

        fulltext_results = graph.query(fulltext_query, {"query": question})

        # 전문 검색 결과의 ID 추가
        for record in fulltext_results:
            if record["id"] and record["id"] not in doc_ids:
                doc_ids.append(record["id"])

        # 검색 결과가 없는 경우 처리
        if not doc_ids:
            return {
                "query": question,
                "result": "관련 법률 조항을 찾을 수 없습니다. 다른 질문을 시도해보세요.",
                "intermediate_steps": [],
            }

        # 3. 그래프 검색: 가변 경로를 사용하여 관련 조문과 그 전후 조문 찾기 (2단계 깊이까지)
        cypher_query = """
        // 검색 결과와 일치하는 법률 조문 찾기
        MATCH (article:Article)
        WHERE article.id IN $doc_ids OR article.title IN $doc_ids

        // 가변 경로를 사용하여 1~2단계 깊이의 조문 찾기 (양방향 고려)
        MATCH path = (article)-[r*1..2]-(related:Article)
        WHERE article <> related  // 자기 자신과의 관계 제외

        // 결과 수집 및 가공
        WITH article, related, r, path
        WITH article,
             related,
             size(r) AS path_length,
             [rel IN r | type(rel)] AS relationship_types

        // 최종 결과 반환
        RETURN article.id AS article_id,
               article.title AS title,
               article.content AS content,
               COLLECT(DISTINCT {
                   id: related.id,
                   title: related.title,
                   content: related.content,
                   path_length: path_length,
                   relationships: relationship_types
               }) AS relatedArticles
        """

        # 그래프 검색 실행 및 결과 처리
        graph_results = graph.query(cypher_query, {"doc_ids": doc_ids})

        # 4. 그래프 정보를 텍스트로 변환
        kg_context = ""
        for record in graph_results:
            kg_context += f"조문: {record['title']}\n내용: {record['content']}\n\n"

            # 관련 조문 정보 추가 (경로 길이에 따라 구분)
            kg_context += "관련 조문:\n"

            # 1단계와 2단계 관계를 구분하여 표시
            level1_articles = [rel for rel in record["relatedArticles"] if rel["path_length"] == 1]
            level2_articles = [rel for rel in record["relatedArticles"] if rel["path_length"] == 2]

            # 1단계 관련 조문 표시
            kg_context += "직접 관련 조문:\n"
            for article in level1_articles:
                rel_type = article["relationships"][0] if article["relationships"] else "관계 없음"
                kg_context += f"- {article['title']} (관계: {rel_type})\n  {article['content']}\n\n"

            # 2단계 관련 조문 표시
            kg_context += "간접 관련 조문:\n"
            for article in level2_articles:
                rel_types = (
                    " -> ".join(article["relationships"])
                    if article["relationships"]
                    else "관계 없음"
                )
                kg_context += (
                    f"- {article['title']} (관계 경로: {rel_types})\n  {article['content']}\n\n"
                )

        # 5. 원본 문서 내용 가져오기
        doc_context = "\n".join(
            [f"{doc.metadata.get('title', '제목 없음')}: {doc.page_content}" for doc in vector_docs]
        )

        # 6. 통합 컨텍스트 생성
        combined_context = f"법률 문서 정보:\n{doc_context}\n\n법률 지식 그래프 정보:\n{kg_context}"

        # 7. 프롬프트 템플릿 정의
        kg_template = """
        당신은 근로기준법에 대한 전문 지식을 갖춘 법률 전문가입니다.
        제공된 법률 조문과 지식 그래프 정보를 바탕으로 질문에 정확하게 답변해 주세요.

        지식 그래프는 법률 조문 간의 관계와 연결성을 보여줍니다.
        이 관계 정보를 활용하여 더 정확하고 포괄적인 법률 해석을 제공하세요.

        법률 조문이나 지식 그래프에서 찾을 수 없는 정보에 대해서는 솔직하게 모른다고 답변하세요.
        답변은 법률 용어를 적절히 사용하되, 일반인도 이해할 수 있도록 명확하게 작성해 주세요.

        참고할 정보:
        {context}

        질문: {question}

        답변:
        """

        kg_prompt = PromptTemplate(template=kg_template, input_variables=["context", "question"])

        # 8. RAG 체인 구성 및 실행
        rag_chain = kg_prompt | llm | StrOutputParser()
        result = rag_chain.invoke({"question": question, "context": combined_context})

        return {
            "query": question,
            "result": result,
            "intermediate_steps": [
                {"vector_search": [doc.metadata for doc in vector_docs]},
                {"fulltext_search": [dict(record) for record in fulltext_results]},
                {"graph_search": [dict(record) for record in graph_results]},
            ],
        }

    except Exception as e:
        return {
            "query": question,
            "result": f"검색 중 오류가 발생했습니다: {str(e)}",
            "error": str(e),
        }


# 실행 테스트
result = hybrid_kg_enhanced_law_rag(
    "법정 근로시간 연장이 가능한 특별한 사정에 대해서 설명해주세요."
)
print(result["result"])
pprint(result["intermediate_steps"])

#### 언제 GraphRAG를 선택할까요?

**적합한 시나리오**:
1. 법률 문서 검색 시스템
2. 기술 문서 (API 문서, 매뉴얼)
3. 학술 논문 검색 (인용 관계)
4. 기업 규정 및 정책 문서
5. 의료 가이드라인 (프로토콜, 절차)

**선택 기준**:
- 문서 간 명확한 관계가 존재
- 계층 구조가 중요한 경우
- 높은 정확도가 요구되는 경우
- 장기적 운영이 예상되는 경우

**GraphRAG의 장점**:
1. 문서 간 관계와 계층 구조 활용
2. 문맥 확장을 통한 정확한 답변
3. 법률/기술 문서에서 매우 높은 정확도
4. Hybrid Search로 다양한 검색 전략 결합 가능


**GraphRAG의 단점**:
1. 그래프 설계 및 구축에 높은 비용
2. 비구조화 문서에 적용 어려움
3. 그래프 유지보수 필요
4. 초기 학습 곡선 높음

#### GraphRAG 구현 시

- 문서 구조 분석 => 구조화된 문서가 있어야합니다. (또는 구조화까지 LLM 에게 명확하게 지시할 수 있어야 합니다.)
- 그래프 스키마 설계 (노드 타입, 관계 타입)
- Neo4j 인프라 구축 (AuraDB 또는 Local)
- 문서 파싱 도구 선택 (Docling, PyMuPDF 등)
- 벡터 인덱스 생성 전략
- Hybrid Search 구현 (Vector + Graph + Fulltext)
- Cypher 쿼리 최적화
- 그래프 업데이트 프로세스 정의